# Capstone: Detection of Products in a Shelf

## Imports

In [ ]:
import os
import subprocess
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
import numpy as np

## Paths

These are the paths to all the relevant folders.

In [ ]:
tf_wsp_path = os.environ.get('CAPSTONE_PATH')+'/TensorFlow/workspace/training_products_shelf/'
annotations_path = tf_wsp_path+'annotations'
images_path = tf_wsp_path+'images'
config_path = tf_wsp_path+'models/my_ssd_resnet50_v1_fpn/pipeline.config'

## Label Map

In [ ]:
labels = [new for original, new in np.load('object_relabelling.npy', allow_pickle=True)]

In [ ]:
counter = 0
label_ls = []
for label in labels:
    counter += 1
    dict = {}
    dict['name'] = label
    dict['id'] = counter
    label_ls.append(dict)

Now we turn the list into the format that the object detection library requires.

In [ ]:
with open(annotations_path+'\label_map.pbtxt', 'w') as f:
    for label in label_ls:
        f.write('item{\n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

First, we need to move to the folder that contains the script that generates the tf-records-files. To generate the train-data-records-file, we run the following code. Note: It took me quite a while to figure out how the slashes should look like.

In [ ]:
slashed_images_path = images_path.replace('/', '\\')
slashed_annotations_path = annotations_path.replace('/', '\\')
subprocess.run('cd TensorFlow\scripts\preprocessing && python generate_tfrecord.py -x '+slashed_images_path+'\\train -l '+slashed_annotations_path+'\\label_map.pbtxt -o '+slashed_annotations_path+'\\train.record', shell=True)

The test-data-record-file is generated by running the following code.

In [ ]:
subprocess.run('cd TensorFlow\scripts\preprocessing && python generate_tfrecord.py -x '+slashed_images_path+'\\test -l '+slashed_annotations_path+'\\label_map.pbtxt -o '+slashed_annotations_path+'\\test.record', shell=True)

## Pipeline Configuration

First, we download a pre-trained model from `https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md`. We extract everything into TensorFlow/workspace/training_products_shelf/pre-trained_models.

Now we copy the pipline.config into TensorFlow/workspace/training_products_shelf/models/my_ssd_resnet50_v1_fpn (or another folder name if we downloaded a different pre-trained model).

Finally, we change the num_classes, batch_size, fine_tune_checkpoint, fine_tune_checkpoint_type, (use_bfloat16), label_map_path, input_path, label_map_path, and input_path. We only need to change use_bfloat16 if we train on a TPU.

In [ ]:
config = config_util.get_configs_from_pipeline_file(config_path)
config

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(config_path, 'r') as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [ ]:
pipeline_config.model.ssd.num_classes = len(labels) # number of different labels
pipeline_config.train_config.batch_size = 8 # larger batch sizes require more memory
pipeline_config.train_config.fine_tune_checkpoint = tf_wsp_path+'/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0' # path to checkpoint of pre-trained model
pipeline_config.train_config.fine_tune_checkpoint_type = 'detection'
pipeline_config.train_input_reader.label_map_path = annotations_path+'/label_map.pbtxt' # path to label map
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [annotations_path+'/train.record'] # path to training-tf-record-file
pipeline_config.eval_input_reader[0].label_map_path = annotations_path+'/label_map.pbtxt' # path to label map
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [annotations_path+'/test.record'] # path to test-tf-record-file

In [ ]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(config_path, 'wb') as f:
    f.write(config_text)

## Model Training

First, we copy TensorFlow/models/research/object_detection/model_main_tf2.py into TensorFlow/workspace/training_products/shelf.

Second, we run 
> `python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config`

in TensorFlow/workspace/training_products_shelf.

## Model Evaluation

We run
> `python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config --checkpoint_dir=models/my_ssd_resnet50_v1_fpn`

in TensorFlow/workspace/training_products_shelf.